In [ ]:
! pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 75% 61.0M/80.9M [00:00<00:00, 119MB/s] 
100% 80.9M/80.9M [00:00<00:00, 121MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'


with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0--->means negative tweet and 1--->means positive tweet

In [ ]:
port_stem= PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content= re.sub('[^a-zA-Z]','',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=''.join(stemmed_content)

  return stemmed_content


In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['abouttowatchsawivanddrinkalilwin' 'hatermagazineimin'
 'eventhoughitsmyfavouritedrinkithinkitsthevodkaandcokethatwipesmymindallthetimethinkimgonnahavetofindanewdrink'
 ... 'iseagerformondayafternoon'
 'hopeeveryoneandtheirmotherhadagreatdaycantwaittohearwhattheguyshaveinstoretomorrow'
 'ilovewakinguptofolgerstoobadmyvoicewasdeeperthanhi']


In [ ]:
print(X_test)

['mmangenmdoingfineihaventhadmuchtimetochatontwitterhubbyisbackforthesummeramptendstodominatemyfreetim'
 'atahsmayshowwruthkimampgeoffreysanhueza'
 'ishataramaybeitwasonlyabayareathangdammit' ...
 'destinineverthelesshoorayformembersandhaveawonderfulandsafetrip'
 'notfeelingtoowel' 'supersandrothanky']


In [ ]:
vectorizer = TfidfVectorizer()

X_train= vectorizer.fit_transform(X_train)
X_test= vectorizer.fit_transform(X_test)

In [ ]:
print(X_train)

  (0, 4269)	1.0
  (1, 370478)	1.0
  (2, 259878)	1.0
  (3, 544662)	1.0
  (4, 1125692)	1.0
  (5, 183084)	1.0
  (6, 655943)	1.0
  (7, 427969)	1.0
  (8, 372293)	1.0
  (9, 414371)	1.0
  (10, 453206)	1.0
  (11, 946311)	1.0
  (12, 1165280)	1.0
  (13, 334893)	1.0
  (14, 1029544)	1.0
  (15, 1251389)	1.0
  (16, 26694)	1.0
  (17, 620642)	1.0
  (18, 371055)	1.0
  (19, 166384)	1.0
  (20, 939661)	1.0
  (21, 392755)	1.0
  (22, 288391)	1.0
  (23, 202689)	1.0
  (24, 1253472)	1.0
  :	:
  (1279975, 97244)	1.0
  (1279976, 1110776)	1.0
  (1279977, 938969)	1.0
  (1279978, 783480)	1.0
  (1279979, 800934)	1.0
  (1279980, 865418)	1.0
  (1279981, 587370)	1.0
  (1279982, 952975)	1.0
  (1279983, 412833)	1.0
  (1279984, 81684)	1.0
  (1279985, 59066)	1.0
  (1279986, 1015971)	1.0
  (1279987, 777243)	1.0
  (1279988, 1218709)	1.0
  (1279989, 450972)	1.0
  (1279990, 465151)	1.0
  (1279991, 249222)	1.0
  (1279992, 1169847)	1.0
  (1279993, 1170137)	1.0
  (1279994, 75555)	1.0
  (1279995, 1106989)	1.0
  (1279996, 561111)	1

In [ ]:
print(X_test)

  (0, 193539)	1.0
  (1, 15208)	1.0
  (2, 131537)	1.0
  (3, 75524)	1.0
  (4, 106316)	1.0
  (5, 108758)	1.0
  (6, 127957)	1.0
  (7, 301475)	1.0
  (8, 199386)	1.0
  (9, 112690)	1.0
  (10, 32110)	1.0
  (11, 74325)	1.0
  (12, 33926)	1.0
  (13, 215744)	1.0
  (14, 304704)	1.0
  (15, 146801)	1.0
  (16, 251714)	1.0
  (17, 35333)	1.0
  (18, 304368)	1.0
  (19, 54669)	1.0
  (20, 171971)	1.0
  (21, 282298)	1.0
  (22, 34327)	1.0
  (23, 313179)	1.0
  (24, 280164)	1.0
  :	:
  (319975, 173561)	1.0
  (319976, 151782)	1.0
  (319977, 27990)	1.0
  (319978, 248013)	1.0
  (319979, 98636)	1.0
  (319980, 315946)	1.0
  (319981, 137258)	1.0
  (319982, 243642)	1.0
  (319983, 116305)	1.0
  (319984, 152248)	1.0
  (319985, 14252)	1.0
  (319986, 50822)	1.0
  (319987, 306608)	1.0
  (319988, 315360)	1.0
  (319989, 2060)	1.0
  (319990, 126089)	1.0
  (319991, 303395)	1.0
  (319992, 143608)	1.0
  (319993, 156126)	1.0
  (319994, 98973)	1.0
  (319995, 278420)	1.0
  (319996, 269151)	1.0
  (319997, 52661)	1.0
  (319998, 21089

training machine learing model
logistic regression

In [ ]:
model= LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model evaluation


Accuracy score


In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.99812265625


In [ ]:
model.fit(X_test, Y_test)

LogisticRegression(max_iter=1000)

In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score on the training data:',test_data_accuracy)

Accuracy score on the training data: 0.999115625


model data accuracy is more than 98%

saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

using the saved model for future predctuion

In [ ]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('negative tweet')
else:
  print('positive tweet')

NameError: name 'X_test' is not defined